Setting up the connection to _Google Drive_.

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Read the _Spreadsheet_'s values.

In [2]:
worksheet = gc.open('Tracking Giornata').sheet1

rows = worksheet.get_all_values()
print(rows[:15])

[['ID', 'Data', 'Orario', 'Azione', 'Categoria', 'Minuti', 'Durata'], ['10000000', '16/09/2023', '8:30:00', 'Sveglia', 'Passive', '510', '10'], ['10000001', '16/09/2023', '8:40:00', 'Colazione', 'Food', '520', '20'], ['10000002', '16/09/2023', '9:00', 'Seguire Sofia mentre si prepara', 'Passive', '540', '45'], ['10000003', '16/09/2023', '9:45', 'Partenza per Rovigo', 'Travelling', '585', '55'], ['10000004', '16/09/2023', '10:40', 'Partenza da Rovigo', 'Travelling', '640', '65'], ['10000005', '16/09/2023', '11:45', 'Arrivo a casa e ordine camera', 'Other', '705', '35'], ['10000006', '16/09/2023', '12:20', 'Relax', 'Passive', '740', '25'], ['10000007', '16/09/2023', '12:45', 'Preparazione pranzo e pranzo', 'Food', '765', '30'], ['10000008', '16/09/2023', '13:15', 'Relax', 'Passive', '795', '135'], ['10000009', '16/09/2023', '15:30', 'Pisolino', 'Sleeping', '930', '35'], ['10000010', '16/09/2023', '16:05', 'Gaming', 'Passive', '965', '55'], ['10000011', '16/09/2023', '17:00', 'Pausetta', 

Save the data in a DataFrame.

In [3]:
import pandas as pd
df_raw = pd.DataFrame.from_records(rows)
df_raw.head(5)

,0,1,2,3,4,5,6
0,ID,Data,Orario,Azione,Categoria,Minuti,Durata
1,10000000,16/09/2023,8:30:00,Sveglia,Passive,510,10
2,10000001,16/09/2023,8:40:00,Colazione,Food,520,20
3,10000002,16/09/2023,9:00,Seguire Sofia mentre si prepara,Passive,540,45
4,10000003,16/09/2023,9:45,Partenza per Rovigo,Travelling,585,55


Adjusting the columns' names.

In [4]:
df_raw.columns = df_raw.iloc[0]
df = df_raw.drop(df_raw.index[0]).reset_index(drop=True)
df.head(5)

,ID,Data,Orario,Azione,Categoria,Minuti,Durata
0,10000000,16/09/2023,8:30:00,Sveglia,Passive,510,10
1,10000001,16/09/2023,8:40:00,Colazione,Food,520,20
2,10000002,16/09/2023,9:00,Seguire Sofia mentre si prepara,Passive,540,45
3,10000003,16/09/2023,9:45,Partenza per Rovigo,Travelling,585,55
4,10000004,16/09/2023,10:40,Partenza da Rovigo,Travelling,640,65


Adjusting columns' type.

In [5]:
df['Minuti'] = pd.to_numeric(df['Minuti'])
df['Durata'] = pd.to_numeric(df['Durata'])
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')

The "Data" column is not enough to specify a day. We consider that the day ends when we go to bed at night (val. "Nanna") and then wake up in the morning (val. "Sveglia"); this bedtime may already have passed midnight. We want to create a column `Phase` that indicates if an action was in a _vacation_ or _school_ day. In a list called `slot_final_index`, we want to insert all the indexes where a new day begins, i.e. track when woke up.

In [6]:
indexes = list(df[(df['Azione'] == 'Sveglia') | (df['Azione'] == 'Nanna')].index)
slot_final_index = [0]
for i in range(len(indexes) - 1):
  if indexes[i] == indexes[i+1] - 1:
    slot_final_index.append(indexes[i+1])
slot_final_index

[0, 16, 38, 59, 82, 97, 114]

Now let's create the list with value "v" for _vacation_ and "s" for _school_, checking that the central index of `slot_final_index`.

In [7]:
phase = ['v' for n in range(slot_final_index[int(len(slot_final_index) / 2)])]

phase = phase + ['s' for n in range(len(phase), len(df))]

df['Phase'] = phase
df

,ID,Data,Orario,Azione,Categoria,Minuti,Durata,Phase
0,10000000,2023-09-16,8:30:00,Sveglia,Passive,510,10,v
1,10000001,2023-09-16,8:40:00,Colazione,Food,520,20,v
2,10000002,2023-09-16,9:00,Seguire Sofia mentre si prepara,Passive,540,45,v
3,10000003,2023-09-16,9:45,Partenza per Rovigo,Travelling,585,55,v
4,10000004,2023-09-16,10:40,Partenza da Rovigo,Travelling,640,65,v
...,...,...,...,...,...,...,...,...
110,005ea013,2023-09-21,20:43:00,Relax,Passive,8443,77,s
111,f6cdcdfa,2023-09-21,22:00:00,Studio,Productivity,8520,210,s
112,1ea7c67b,2023-09-22,01:30:00,Consolazione Sofi,Passive,8730,30,s
113,f35cec9b,2023-09-22,02:00:00,Nanna,Sleeping,8760,345,s


Grouping and sum by "Durata".

In [8]:
df_grouped = df.groupby(['Phase', 'Categoria'])['Durata'].sum().reset_index(name='sum')
df_grouped

,Phase,Categoria,sum
0,s,Food,384
1,s,Other,268
2,s,Passive,776
3,s,Productivity,1223
4,s,Sleeping,1244
5,s,Travelling,400
6,v,Food,400
7,v,Other,220
8,v,Passive,2125
9,v,Productivity,120


Pivot the column _Phase_

In [9]:
df_pivot_raw = df_grouped.pivot(index='Categoria', columns='Phase', values='sum')
df_pivot_raw

Phase,s,v
Categoria,,
Food,384,400
Other,268,220
Passive,776,2125
Productivity,1223,120
Sleeping,1244,1215
Travelling,400,220


In [10]:
df_pivot = pd.DataFrame({'Category' : list(df_pivot_raw.index),
                         'Vacation' : list(df_pivot_raw['v']),
                         'School' : list(df_pivot_raw['s'])},
                         index=range(len(df_pivot_raw)))
df_pivot

,Category,Vacation,School
0,Food,400,384
1,Other,220,268
2,Passive,2125,776
3,Productivity,120,1223
4,Sleeping,1215,1244
5,Travelling,220,400


Taking the pct of every _Category_

In [11]:
df_pivot['Vacation'] = round(df_pivot['Vacation'] / df_pivot['Vacation'].sum() * 100, 1)
df_pivot['School'] = round(df_pivot['School'] / df_pivot['School'].sum() * 100, 1)
df_pivot

,Category,Vacation,School
0,Food,9.3,8.9
1,Other,5.1,6.2
2,Passive,49.4,18.1
3,Productivity,2.8,28.5
4,Sleeping,28.3,29.0
5,Travelling,5.1,9.3
